In [1]:
import numpy as np
import pandas as pd
import random, pickle, argparse, json, os, urllib2
from collections import OrderedDict
from operator import itemgetter
from sklearn.ensemble import RandomForestClassifier

In [2]:
def query_from(q, f):
    q = q+'&from='+str(f)
    response = urllib2.urlopen(q)
    data = json.load(response)
    subject_ids = np.array(range(len(data['hits'])), dtype=np.object)
    for (i, hit) in enumerate(data['hits']):
        subject_ids[i] = hit['id']
    return subject_ids

def query(q):
    response = urllib2.urlopen(q)
    data = json.load(response)
    nb_requests = 1 + data['total'] / 1000
    if nb_requests > 10: # maximum number of pages due to API pagination restrection
        nb_requests = 10
    subject_ids = query_from(q, 0)
    for i in range(nb_requests)[1:]:
        f = i * 1000
        next_request = query_from(q, f)
        subject_ids = np.hstack((subject_ids, next_request))
    return subject_ids.tolist()

In [3]:
def find_intersection(list_a, list_b):
    return list(set(list_a) & set(list_b))
a = ['hussein', 'tayseer', 'al-natsheh']
b = ['loay', 'hussein', 'al-natsheh']
find_intersection(a, b)

['al-natsheh', 'hussein']

In [4]:
inv_index = json.load(open('../RecSys_Exp_files/182_381_vec150_results/output_paragraph_inversed_index.json','rb'))
print 'original inversed_index'
print inv_index.items()[:3]
inversed_index = dict()
for (k, v) in inv_index.items():
    key = k.split('_')[1]
    inversed_index[key] = v
print 'processed inversed_index'
print inversed_index.items()[:3]

original inversed_index
[(u'ISTEX_D89FA3AC3521074D46F4245762153DF497BFFA1F', 2002320), (u'ISTEX_18EAF4D6A126B077EB38667801D1B7292F32FF49', 2483732), (u'ISTEX_5F91044435FCC4FABB9F02E31467DCFE75F4A7BE', 1429049)]
processed inversed_index
[(u'FCF1393F9B8136AC08FB67E88F94F3CF62C17288', 3517138), (u'482E1102A1114327A744FD2ADB4D9F8FF7E9A70B', 751643), (u'A81022B6295AE66F68A10222C3B94A06B033C1BA', 3983232)]


In [5]:
def babelnet_get_input():
    results = query('https://api.istex.fr/document/?q=((title:%22Biodiversity%20Conservation%22%20OR%20abstract:%22Biodiversity%20Conservation%22%20OR%20title:%22conservation%20biology%22%20OR%20abstract:%22conservation%20biology%22%20OR%20title:%22Animal%20conservation%22%20OR%20abstract:%22Animal%20conservation%22%20OR%20title:%22Biological%20Conservation%22%20OR%20abstract:%22Biological%20Conservation%22%20ORtitle:%22Conservation%20of%20natural%20resources%22%20OR%20abstract:%22Conservation%20of%20natural%20resources%22%20OR%20title:%22Conservation%20of%20wildlife%22%20OR%20abstract:%22Conservation%20of%20wildlife%22%20ORtitle:%22Wildlife%20Conservation%22%20OR%20abstract:%22Wildlife%20Conservation%22%20OR%20title:%22Ecological%20conservation%22%20OR%20abstract:%22Ecological%20conservation%22%20ORtitle:%22Conservation%20science%22%20OR%20abstract:%22Conservation%20science%22%20OR%20title:%22Conservation%22%20OR%20abstract:%22Conservation%22%20ORtitle:%22Conservation%20biologist%22%20OR%20abstract:%22Conservation%20biologist%22%20OR%20title:%22Conservation%20biologists%22%20OR%20abstract:%22Conservation%20biologists%22%20ORtitle:%22Conservation%20priority%22%20OR%20abstract:%22Conservation%20priority%22%20OR%20title:%22Earth%20biologist%22%20OR%20abstract:%22Earth%20biologist%22%20ORtitle:%22Earth%20biology%22%20OR%20abstract:%22Earth%20biology%22%20OR%20title:%22History%20of%20conservation%20biology%22%20OR%20abstract:%22History%20of%20conservation%20biology%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    _gs = query('https://api.istex.fr/document/?q=((subject.value:%22Biodiversity%20Conservation%22%20OR%20keywords:%22Biodiversity%20Conservation%22%20)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    results = find_intersection(results, inversed_index.keys())
    _abst_title = query('https://api.istex.fr/document/?q=((title:%22Biodiversity%20Conservation%22%20OR%20abstract:%22Biodiversity%20Conservation%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id')
    test_set = _inter = {x for x in _gs if x not in _abst_title}
    test_set = find_intersection(test_set, inversed_index.keys())
    results = list(results)
    test = list(test_set)
    print 'initial_corpus size:', len(find_intersection(_abst_title, inversed_index.keys()))
    return results, test
babelnet_results, test = babelnet_get_input()
print len(babelnet_results), len(test)

initial_corpus size: 344
7488 55


In [5]:
def term2url(string):
    string = string.split(' ')
    res = '%22'
    for s in string:
        res = res + s + '%20'
    res = res[:-3]
    res = res + '%22'
    return res
term2url('conservation biology')

'%22conservation%20biology%22'

In [6]:
use_synset = "Emergency medicine, Emergency care, Emergency med, Emergency treatment, Emergent condition, Emergentology, Er physician"
synset_t = use_synset.split(', ')
synset = []
for syn in synset_t:
    x = syn.replace(' ','%20')
    synset.append(x)
print synset

['Emergency%20medicine', 'Emergency%20care', 'Emergency%20med', 'Emergency%20treatment', 'Emergent%20condition', 'Emergentology', 'Er%20physician']


In [8]:
synset_text = 'Sports science, Sport science, Sport Sciences, Human movement, Human movement studies, Science and sports, Sport and Exercise Science, Sport scientist, Sports sciences, Sports scientist, Sports scientists'
synset = synset_text.split(',')
synset

['Sports science',
 ' Sport science',
 ' Sport Sciences',
 ' Human movement',
 ' Human movement studies',
 ' Science and sports',
 ' Sport and Exercise Science',
 ' Sport scientist',
 ' Sports sciences',
 ' Sports scientist',
 ' Sports scientists']

In [7]:
def babel_synset(synset):
    q = 'https://api.istex.fr/document/?q=(('
    for syn in synset:
        syn = term2url(syn)
        q = q + 'title:' + syn + '%20OR%20abstract:' + syn + '%20OR%20'
    q = q[:-8]
    q = q + ')%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
    return q

#test babel_synset
#synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
#print babel_synset(synset)
#print 'https://api.istex.fr/document/?q=((title:%22conservation%20biology%22%20OR%20abstract:%22conservation%20biology%22%20ORtitle:%22Animal%20conservation%22%20OR%20abstract:%22Animal%20conservation%22%20OR%20title:%22Biological%20Conservation%22%20OR%20abstract:%22Biological%20Conservation%22%20ORtitle:%22Conservation%20of%20natural%20resources%22%20OR%20abstract:%22Conservation%20of%20natural%20resources%22%20OR%20title:%22Conservation%20of%20wildlife%22%20OR%20abstract:%22Conservation%20of%20wildlife%22%20ORtitle:%22Wildlife%20Conservation%22%20OR%20abstract:%22Wildlife%20Conservation%22%20OR%20title:%22Ecological%20conservation%22%20OR%20abstract:%22Ecological%20conservation%22%20ORtitle:%22Conservation%20science%22%20OR%20abstract:%22Conservation%20science%22%20OR%20title:%22Conservation%22%20OR%20abstract:%22Conservation%22%20ORtitle:%22Conservation%20biologist%22%20OR%20abstract:%22Conservation%20biologist%22%20OR%20title:%22Conservation%20biologists%22%20OR%20abstract:%22Conservation%20biologists%22%20ORtitle:%22Conservation%20priority%22%20OR%20abstract:%22Conservation%20priority%22%20OR%20title:%22Earth%20biologist%22%20OR%20abstract:%22Earth%20biologist%22%20ORtitle:%22Earth%20biology%22%20OR%20abstract:%22Earth%20biology%22%20OR%20title:%22History%20of%20conservation%20biology%22%20OR%20abstract:%22History%20of%20conservation%20biology%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'

def babel_subj_keyword(topic):
    q = 'https://api.istex.fr/document/?q=(('
    topic = term2url(topic)
    q = q+ 'subject.value:' + topic + '%20OR%20keywords:' + topic
    q = q + ')%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
    return q
 
#test babel_subj_keyword
#print babel_subj_keyword('Biodiversity Conservation')
#print 'https://api.istex.fr/document/?q=((subject.value:%22Biodiversity%20Conservation%22%20OR%20keywords:%22Biodiversity%20Conservation%22%20)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'

def babel_title_abst(topic):
    q = 'https://api.istex.fr/document/?q=(('
    topic = term2url(topic)
    q = q+ 'title:' + topic + '%20OR%20abstract:' + topic
    q = q + ')%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'
    return q
 
#test babel_title_abst
#print babel_title_abst('Biodiversity Conservation')
#print 'https://api.istex.fr/document/?q=((title:%22Biodiversity%20Conservation%22%20OR%20abstract:%22Biodiversity%20Conservation%22)%20AND%20(qualityIndicators.abstractWordCount:[35%20500]%20AND%20qualityIndicators.pdfPageCount:[3%2060]%20AND%20publicationDate:[1990%202016]%20AND%20language:(%22eng%22%20OR%20%22unknown%22)%20AND%20genre:(%22research_article%22%20OR%20%22conference[eBooks]%22%20OR%20%22article%22%20)%20))&size=1000&output=id'

In [8]:
def babelnet_syn_get_input(topic, synset):
    results = query(babel_synset(synset))
    _gs = query(babel_subj_keyword(topic))
    results = find_intersection(results, inversed_index.keys())
    _abst_title = query(babel_title_abst(topic))
    test_set = _inter = {x for x in _gs if x not in _abst_title}
    test_set = find_intersection(test_set, inversed_index.keys())
    results = list(results)
    test = list(test_set)
    print 'initial_corpus size:', len(find_intersection(_abst_title, inversed_index.keys()))
    return results, test
#test babelnet_syn_get_input
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
babelnet_results, test = babelnet_syn_get_input('Biodiversity Conservation', synset)
print len(babelnet_results), len(test)

initial_corpus size: 344
7481 55


In [11]:
cond = pickle.load(open('results/res__Condensed_matter','rb'))

In [16]:
cond[:10]

[(u'269044D0F0C5DD5F37043AA209B72F7DA99003FE', 0.98799999999999999),
 (u'421D402518F03D805E8A29EF04D4BE93539F1FA7', 0.97999999999999998),
 (u'658F818CB292894E38B02C72EA9BA602AA47A587', 0.97599999999999998),
 (u'E189B02A80A7DD0E49D85098A0ABD9D3DEB78F62', 0.97599999999999998),
 (u'E95A15FB563CC5A94E27EFEA16BB4B9F7C71313B', 0.97599999999999998),
 (u'2FB037B74E1B24E4BD7EC87F46D04CCCB373D212', 0.97399999999999998),
 (u'6A3390346932D416990B07583AFD6D44AE91A219', 0.97399999999999998),
 (u'466EAC8621A99784C5F6EEEF54C6DED85D4D97F9', 0.97199999999999998),
 (u'D550E2F31532FA9D0D5211CE584998B69BCE3CC6', 0.96999999999999997),
 (u'776E1BBE47292EF9047FF0671A9DF0D6B693ABCE', 0.96999999999999997)]

In [10]:
s3h_results = pickle.load(open('results/BC0t_results.pickle', 'rb'))

In [44]:
s3h_syn_results = pickle.load(open('results/BC0t_1_RFC_syn_results.pickle', 'rb'))

In [15]:
len(test)

55

In [18]:
babel_test_intersection = find_intersection(test,babelnet_results)
babel_test_intersection_size = len(babel_test_intersection)
print babel_test_intersection_size

12


In [9]:
def babelnet_eval(topic, synset):
    babelnet_results, test = babelnet_syn_get_input(topic, synset)
    print 'results size of the topic "' + topic + '":', len(babelnet_results) 
    print 'ground truth size', len(test)
    babel_test_intersection = find_intersection(test,babelnet_results)
    babel_test_intersection_size = len(babel_test_intersection)
    print 'intersection with the ground truth:', babel_test_intersection_size

In [26]:
synset = ['Remote sensing', 'Infrared remote sensing', 'Passive remote sensing', 'Remote-sensing', 'Remote-Sensing Image', 'Remote Sensing Satellites', 'Remote sensor']
topic = 'Remote sensing'
babelnet_eval(topic, synset)

initial_corpus size: 4642
results size of the topic "Remote sensing": 4685
ground truth size 1192
intersection with the ground truth: 3


In [99]:
agri = 'agricultural engineering, agrotechnology, Farm Engineering, Agricultural engineer, Agricultural engineers, Agriculture Engineering'
synset = agri.split(',')
topic = 'agricultural engineering'
babelnet_eval(topic, synset)

initial_corpus size: 20
results size of the topic "agricultural engineering": 46
ground truth size 5
intersection with the ground truth: 0


In [27]:
synset = ['Respiratory system', 'systema respiratorium', 'Respiration organ', 'Breathing organ', 'Breathing system', 'Development of respiratory system', 'Human Respiration', 'Human respiratory system', 'Pulmonary respiration', 'Pulmonary system', 'Respatory system', 'Respiration of human', 'Respiration system, Respiratory', 'Respiratory organs', 'Respiratory system agents', 'Respiratory system disorders', 'Respiratory systems', 'Respitory System', 'Subglottic airway', 'The respiratory system']
topic = 'Respiratory system'
babelnet_eval(topic, synset)

initial_corpus size: 795
results size of the topic "Respiratory system": 1024
ground truth size 189
intersection with the ground truth: 3


In [28]:
synset = ['Emergency medicine', 'Emergency care', 'Emergency med', 'Emergency treatment', 'Emergent condition', 'Emergentology', 'Er physician']
topic = 'Emergency medicine'
babelnet_eval(topic, synset)

initial_corpus size: 810
results size of the topic "Emergency medicine": 1379
ground truth size 285
intersection with the ground truth: 14


In [29]:
synset = ['Substance abuse', 'drug abuse', 'habit', 'addiction', 'dependency', 'Abuse potential', 'Abusing drugs', 'Abusive drug use', 'Anti-drug', 'Cannabis abuse', 'Drug-abuse', 'Drug misuse', 'Drug prevention', 'Drugs of abuse', 'Illegal drug abuse', 'Illegal drug use', 'Misuse of drugs', 'Narcotic abuse theory', 'Nondependent abuse of drugs', 'Prescription drug abuse', 'Prescription Drug Misuse']
topic = 'Substance abuse'
babelnet_eval(topic, synset)

initial_corpus size: 2658
results size of the topic "Substance abuse": 7893
ground truth size 466
intersection with the ground truth: 66


In [30]:
synset = ['international relations', 'international affairs', 'world affairs', 'International politics', 'Intrel', 'Diplomacy and Statecraft', 'Diplomatic relationship', 'Global relations', 'International relation', 'International Relation Studies', 'International trade relations', 'Interstate relations', 'Levels of Analysis in international relations', 'Study of International Relations']
topic = 'international relations'
babelnet_eval(topic, synset)

initial_corpus size: 724
results size of the topic "international relations": 983
ground truth size 60
intersection with the ground truth: 5


In [32]:
synset = ['forensic computing', 'Digital forensics', 'Digital forensic science', 'Evidence collection', 'Forensic computing', 'Intrusion investigation']
topic = 'Digital forensics'
babelnet_eval(topic, synset)

initial_corpus size: 3
results size of the topic "Digital forensics": 34
ground truth size 0
intersection with the ground truth: 0


In [13]:
ranked_all = pickle.load(open('results/Substance_results.pickle', 'rb'))

In [16]:
ranked_all_np = np.array(ranked_all.items())

In [60]:
top50 = ranked_all_np[:50]
#top50

In [51]:
pd.DataFrame?

In [63]:
top50_df = pd.DataFrame(data=top50, index=None, columns=['istex_id', 'score'], copy=False)
top50_df['score'] = top50_df[['score']].astype(float)
top50_df[top50_df['score'] > 0.988]

,istex_id,score
0,6E940E7B1FEE274AE69561BD9B2C6FE55BF45951,0.99
1,852F700B181D092A2FAC5D405B51824FC49EA929,0.99


In [67]:
def top_thresh(ordered_dict_pickle, thresh):
    ranked_all = pickle.load(open(ordered_dict_pickle, 'rb'))
    ranked_all_np = np.array(ranked_all.items())
    ranked_all_df = pd.DataFrame(data=ranked_all_np, index=None, columns=['istex_id', 'score'])
    ranked_all_df['score'] = ranked_all_df[['score']].astype(float)
    return ranked_all_df[ranked_all_df['score'] > thresh]
top_substance_95 = top_thresh('results/Substance_results.pickle', 0.95)
print len(top_substance_95)

130


In [22]:
def top_thresh_lst(res_lst_pickle, thresh):
    ranked_all = pickle.load(open(res_lst_pickle, 'rb'))
    ranked_all_np = np.array(ranked_all)
    ranked_all_df = pd.DataFrame(data=ranked_all_np, index=None, columns=['istex_id', 'score'])
    ranked_all_df['score'] = ranked_all_df[['score']].astype(float)
    return ranked_all_df[ranked_all_df['score'] > thresh]
top_cond_75 = top_thresh_lst('results/res__Condensed_matter', 0.75)
print len(top_cond_75)

9988


In [70]:
top_substance_85 = top_thresh('results/Substance_results.pickle', 0.85)

In [69]:
print len(top_substance_85)

1368


In [65]:
top_respiratory_85 = top_thresh('results/Respiratory_system_results.pickle', 0.85)
print len(top_respiratory_85)

50


In [84]:
#Evaluate 3SH results at treshold
def eval_at_thresh(ordered_dict_pickle, topic, synset, thresh):
    _, test = babelnet_syn_get_input(topic, synset)
    t = len(test)
    top_res = top_thresh(ordered_dict_pickle, thresh)
    n = len(top_res)
    res = list(top_res['istex_id'])
    recall = len(find_intersection(test,res))/float(t)
    precision = len(find_intersection(test,res))/float(n)
    return precision, recall, n, t

In [85]:
topic = 'Respiratory system'
synset = ['Respiratory system', 'systema respiratorium', 'Respiration organ', 'Breathing organ', 'Breathing system', 'Development of respiratory system', 'Human Respiration', 'Human respiratory system', 'Pulmonary respiration', 'Pulmonary system', 'Respatory system', 'Respiration of human', 'Respiration system, Respiratory', 'Respiratory organs', 'Respiratory system agents', 'Respiratory system disorders', 'Respiratory systems', 'Respitory System', 'Subglottic airway', 'The respiratory system']
p , r, n, t = eval_at_thresh('results/Respiratory_system_results.pickle', topic , synset, 0.7)
print '3sh tresh results of'+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 795
3sh tresh results ofRespiratory system
precision: 0.00421274354924 recall: 0.042328042328
number of selected items "3sh results lenght": 1899 lenght of correct answers: 189


In [87]:
topic = 'Substance abuse'
synset = ['Substance abuse', 'drug abuse', 'habit', 'addiction', 'dependency', 'Abuse potential', 'Abusing drugs', 'Abusive drug use', 'Anti-drug', 'Cannabis abuse', 'Drug-abuse', 'Drug misuse', 'Drug prevention', 'Drugs of abuse', 'Illegal drug abuse', 'Illegal drug use', 'Misuse of drugs', 'Narcotic abuse theory', 'Nondependent abuse of drugs', 'Prescription drug abuse', 'Prescription Drug Misuse']
p , r, n, t = eval_at_thresh('results/Substance_results.pickle', topic , synset, 0.7)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 2658
3sh tresh results of Substance abuse
precision: 0.00862325304787 recall: 0.124463519313
number of selected items "3sh results lenght": 6726 lenght of correct answers: 466


In [88]:
topic = 'Remote sensing'
synset = ['Remote sensing', 'Infrared remote sensing', 'Passive remote sensing', 'Remote-sensing', 'Remote-Sensing Image', 'Remote Sensing Satellites', 'Remote sensor']
p , r, n, t = eval_at_thresh('results/Remote_sensing_results.pickle', topic , synset, 0.7)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 4642
3sh tresh results of Remote sensing
precision: 0.0226702111948 recall: 0.515100671141
number of selected items "3sh results lenght": 27084 lenght of correct answers: 1192


In [89]:
synset = ['international relations', 'international affairs', 'world affairs', 'International politics', 'Intrel', 'Diplomacy and Statecraft', 'Diplomatic relationship', 'Global relations', 'International relation', 'International Relation Studies', 'International trade relations', 'Interstate relations', 'Levels of Analysis in international relations', 'Study of International Relations']
topic = 'international relations'
p , r, n, t = eval_at_thresh('results/international_relations_results.pickle', topic , synset, 0.7)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 724
3sh tresh results of international relations
precision: 0.00126830393174 recall: 0.183333333333
number of selected items "3sh results lenght": 8673 lenght of correct answers: 60


In [90]:
synset = ['Emergency medicine', 'Emergency care', 'Emergency med', 'Emergency treatment', 'Emergent condition', 'Emergentology', 'Er physician']
topic = 'Emergency medicine'
p , r, n, t = eval_at_thresh('results/Emergence_results.pickle', topic , synset, 0.7)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 810
3sh tresh results of Emergency medicine
precision: 0.00237529691211 recall: 0.0105263157895
number of selected items "3sh results lenght": 1263 lenght of correct answers: 285


In [91]:
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
topic = 'Biodiversity Conservation'
p , r, n, t = eval_at_thresh('results/Biodiversity_Conservation_results.pickle', topic , synset, 0.7)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 344
3sh tresh results of Biodiversity Conservation
precision: 0.00338218714769 recall: 0.0545454545455
number of selected items "3sh results lenght": 887 lenght of correct answers: 55


In [ ]:
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
topic = 'Biodiversity Conservation'
p , r, n, t = eval_at_thresh('results/Biodiversity_Conservation_results.pickle', topic , synset, 0.7)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

In [92]:
top_bio_65 = top_thresh('results/Biodiversity_Conservation_results.pickle', 0.65)
print len(top_bio_65)
top_bio_60 = top_thresh('results/Biodiversity_Conservation_results.pickle', 0.60)
print len(top_bio_60)
top_bio_55 = top_thresh('results/Biodiversity_Conservation_results.pickle', 0.55)
print len(top_bio_55)

1646
3207
6038


In [94]:
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
topic = 'Biodiversity Conservation'
p , r, n, t = eval_at_thresh('results/Biodiversity_Conservation_results.pickle', topic , synset, 0.6)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 344
3sh tresh results of Biodiversity Conservation
precision: 0.00311817898347 recall: 0.181818181818
number of selected items "3sh results lenght": 3207 lenght of correct answers: 55


In [96]:
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
topic = 'Biodiversity Conservation'
p , r, n, t = eval_at_thresh('results/Biodiversity_Conservation_results.pickle', topic , synset, 0.55)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 344
3sh tresh results of Biodiversity Conservation
precision: 0.00231864855913 recall: 0.254545454545
number of selected items "3sh results lenght": 6038 lenght of correct answers: 55


In [93]:
top_remote_85 = top_thresh('results/Remote_sensing_results.pickle', 0.85)
print len(top_remote_85)
top_remote_80 = top_thresh('results/Remote_sensing_results.pickle', 0.80)
print len(top_remote_80)
top_remote_75 = top_thresh('results/Remote_sensing_results.pickle', 0.75)
print len(top_remote_75)

7655
12323
18560


In [95]:
topic = 'Remote sensing'
synset = ['Remote sensing', 'Infrared remote sensing', 'Passive remote sensing', 'Remote-sensing', 'Remote-Sensing Image', 'Remote Sensing Satellites', 'Remote sensor']
p , r, n, t = eval_at_thresh('results/Remote_sensing_results.pickle', topic , synset, 0.85)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 4642
3sh tresh results of Remote sensing
precision: 0.0335728282169 recall: 0.215604026846
number of selected items "3sh results lenght": 7655 lenght of correct answers: 1192


In [97]:
synset = ['Emergency medicine', 'Emergency care', 'Emergency med', 'Emergency treatment', 'Emergent condition', 'Emergentology', 'Er physician']
topic = 'Emergency medicine'
p , r, n, t = eval_at_thresh('results/Emergence_results.pickle', topic , synset, 0.55)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 810
3sh tresh results of Emergency medicine
precision: 0.00451671183379 recall: 0.0701754385965
number of selected items "3sh results lenght": 4428 lenght of correct answers: 285


In [98]:
topic = 'Respiratory system'
synset = ['Respiratory system', 'systema respiratorium', 'Respiration organ', 'Breathing organ', 'Breathing system', 'Development of respiratory system', 'Human Respiration', 'Human respiratory system', 'Pulmonary respiration', 'Pulmonary system', 'Respatory system', 'Respiration of human', 'Respiration system, Respiratory', 'Respiratory organs', 'Respiratory system agents', 'Respiratory system disorders', 'Respiratory systems', 'Respitory System', 'Subglottic airway', 'The respiratory system']
p , r, n, t = eval_at_thresh('results/Respiratory_system_results.pickle', topic , synset, 0.55)
print '3sh tresh results of'+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 795
3sh tresh results ofRespiratory system
precision: 0.00223435948361 recall: 0.0952380952381
number of selected items "3sh results lenght": 8056 lenght of correct answers: 189


In [172]:
bab = 'artificial intelligence, AI, artilect, Artifical intelligence, Cognitive systems, Digital being, Machine intelligence, A.I., A I, AI-D, AI ethics, AI implications, AI Robotics, AI scripting, Aretificial intelligence, Artificial-intelligence, Artificial conciousness, Artificial inteligence, Artificial intellect, Artificial intellegence, Artificial Intelligence., Artificial intelligence for development, Artificial Intelligence Program, Artificial intelligences, Artificially-intelligent, Artificially intelligent, Artificual intelligence, Cognitive simulation, Cognitive system, Commonsense AI, Computational Rationality, Computer AI, Intelligent machine, Machine thought, Machine understanding, Ontology based approach, Pseudo intelligence, Semi-AI, Semi AI, Simulated intelligence, Soft AI, Sub-symbolic, Subsymbolic, The Artificial Intelligence, The Theory of Artificial Intelligence'
synset = bab.split(',')
topic = 'artificial intelligence'
p , r, n, t = eval_at_thresh('results/AI_results.pickle', topic , synset, 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 2091
3sh tresh results ofartificial intelligence
precision: 0.00484888073763 recall: 0.293759512938
number of selected items "3sh results lenght": 39803 lenght of correct answers: 657


In [173]:
bab = 'information system, data system, information systems, system info, Business computing, Computer information system, Computer information systems, Elements of Information System, Information in Computer Science, Information systems and technology, Information systems discipline, Information systems theory, Informationssystem'
synset = bab.split(',')
topic = 'Information Systems'
p , r, n, t = eval_at_thresh('results/infosys_results.pickle', topic , synset, 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 5033
3sh tresh results ofInformation Systems
precision: 0.0116214720531 recall: 0.0799695354151
number of selected items "3sh results lenght": 9035 lenght of correct answers: 1313


In [19]:
def babelnet_eval_PR(topic, synset):
    babelnet_results, test = babelnet_syn_get_input(topic, synset)
    print 'babelnet results size of the topic "' + topic + '":', len(babelnet_results) 
    print 'ground truth size', len(test)
    babel_test_intersection = find_intersection(test,babelnet_results)
    babel_test_intersection_size = len(babel_test_intersection)
    print 'intersection with the ground truth:', babel_test_intersection_size
    print 'babel precision: ', babel_test_intersection_size / float(len(babelnet_results))
    print 'babel recall: ', babel_test_intersection_size / float(len(test))

In [14]:
#Evaluate 3SH results at treshold
def eval_all_at_thresh(ordered_dict_pickle, topic, synset, thresh):
    babelnet_eval_PR(topic, synset)
    _, test = babelnet_syn_get_input(topic, synset)
    t = len(test)
    top_res = top_thresh(ordered_dict_pickle, thresh)
    n = len(top_res)
    if n > 10000:
        thresh = thresh + 0.1
        top_res = top_thresh(ordered_dict_pickle, thresh)
    elif n < 1000:
        thresh = thresh - 0.1
        top_res = top_thresh(ordered_dict_pickle, thresh)
    n = len(top_res)
    res = list(top_res['istex_id'])
    recall = len(find_intersection(test,res))/float(t)
    precision = len(find_intersection(test,res))/float(n)
    return precision, recall, n, t, thresh

In [24]:
#Evaluate 3SH results list at treshold
def eval_all_at_thresh_lst(res_pickle, topic, synset, thresh=0.75):
    babelnet_eval_PR(topic, synset)
    _, test = babelnet_syn_get_input(topic, synset)
    t = len(test)
    top_res = top_thresh_lst(res_pickle, thresh)
    n = len(top_res)
    if n > 10000:
        thresh = thresh + 0.1
        top_res = top_thresh_lst(res_pickle, thresh)
        if len(top_res) > 10000:
            thresh = thresh + 0.05
            top_res = top_thresh_lst(res_pickle, thresh)
    elif n < 1000:
        thresh = thresh - 0.1
        top_res = top_thresh_lst(res_pickle, thresh)
        if len(top_res) > 1000:
            thresh = thresh - 0.05
            top_res = top_thresh_lst(res_pickle, thresh)
    n = len(top_res)
    res = list(top_res['istex_id'])
    recall = len(find_intersection(test,res))/float(t)
    precision = len(find_intersection(test,res))/float(n)
    return precision, recall, n, t, thresh

In [188]:
bab = 'artificial intelligence, AI, artilect, Artifical intelligence, Cognitive systems, Digital being, Machine intelligence, A.I., A I, AI-D, AI ethics, AI implications, AI Robotics, AI scripting, Aretificial intelligence, Artificial-intelligence, Artificial conciousness, Artificial inteligence, Artificial intellect, Artificial intellegence, Artificial Intelligence., Artificial intelligence for development, Artificial Intelligence Program, Artificial intelligences, Artificially-intelligent, Artificially intelligent, Artificual intelligence, Cognitive simulation, Cognitive system, Commonsense AI, Computational Rationality, Computer AI, Intelligent machine, Machine thought, Machine understanding, Ontology based approach, Pseudo intelligence, Semi-AI, Semi AI, Simulated intelligence, Soft AI, Sub-symbolic, Subsymbolic, The Artificial Intelligence, The Theory of Artificial Intelligence'
synset = bab.split(',')
topic = 'artificial intelligence'
p , r, n, t, thresh = eval_all_at_thresh('results/AI_results.pickle', topic , synset, thresh = 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t
print 'threshold = ', thresh

initial_corpus size: 2091
babelnet results size of the topic "artificial intelligence": 7903
ground truth size 657
intersection with the ground truth: 43
babel precision:  0.00544097178287
babel recall:  0.0654490106545
initial_corpus size: 2091
3sh tresh results of artificial intelligence
precision: 0.00933924819052 recall: 0.121765601218
number of selected items "3sh results lenght": 8566 lenght of correct answers: 657
threshold =  0.85


In [190]:
bab = 'information system, data system, information systems, system info, Business computing, Computer information system, Computer information systems, Elements of Information System, Information in Computer Science, Information systems and technology, Information systems discipline, Information systems theory, Informationssystem'
synset = bab.split(',')
topic = 'Information Systems'
p , r, n, t, thresh = eval_all_at_thresh('results/infosys_results.pickle', topic , synset, thresh = 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t
print 'threshold = ', thresh

initial_corpus size: 5033
babelnet results size of the topic "Information Systems": 8440
ground truth size 1313
intersection with the ground truth: 298
babel precision:  0.035308056872
babel recall:  0.226961157654
initial_corpus size: 5033
3sh tresh results of Information Systems
precision: 0.0104105571848 recall: 0.0540746382331
number of selected items "3sh results lenght": 6820 lenght of correct answers: 1313
threshold =  0.75


In [191]:
bab = 'neuroimaging, Brain imaging, Brain scanning, Brain function map, Brain scan, Brain scans, Functional neurological mapping, Neuro-imaging, Neuroradiography'
synset = bab.split(',')
topic = 'Neuroimaging'
p , r, n, t, thresh = eval_all_at_thresh('results/Neuroimaging_results.pickle', topic , synset, thresh = 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t
print 'threshold = ', thresh

initial_corpus size: 2497
babelnet results size of the topic "Neuroimaging": 3679
ground truth size 522
intersection with the ground truth: 52
babel precision:  0.0141342756184
babel recall:  0.0996168582375
initial_corpus size: 2497
3sh tresh results of Neuroimaging
precision: 0.0153054221002 recall: 0.427203065134
number of selected items "3sh results lenght": 14570 lenght of correct answers: 522
threshold =  0.85


In [192]:
bab = 'infection, infectious disease, communicable diseases, contagion, Infectious diseases, Acute infection, AIDS-related bacterial infections, AIDS-related viral infections, Anti-infective, Anti-infectives, Antiinfective, Bacterial Infections, Communicable disease, Contagious diseases, Definition to contagious, Infect, Infecting, Infectiology, Infections, Infectious, Infectious disease epidemiology, Infectious disease medicine, Infectology, Local infection, Primary infection, Rochalimea infections, Secondary infection, Tropical bacterial infections, Tropical infections, Viral Infections, Wound colonization, Wound infection'
synset = bab.split(',')
topic = 'Infectious Diseases'
p , r, n, t, thresh = eval_all_at_thresh('results/Infectious_results.pickle', topic , synset, thresh = 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t
print 'threshold = ', thresh

initial_corpus size: 2588
babelnet results size of the topic "Infectious Diseases": 8343
ground truth size 1375
intersection with the ground truth: 50
babel precision:  0.00599304806425
babel recall:  0.0363636363636
initial_corpus size: 2588
3sh tresh results of Infectious Diseases
precision: 0.00153295861012 recall: 0.00218181818182
number of selected items "3sh results lenght": 1957 lenght of correct answers: 1375
threshold =  0.75


In [193]:
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
topic = 'Biodiversity Conservation'
p , r, n, t, thresh = eval_all_at_thresh('results/Biodiversity_Conservation_results.pickle', topic , synset, thresh = 0.75)
print '3sh tresh results of '+topic
print 'precision:', p, 'recall:', r
print 'number of selected items "3sh results lenght":', n, 'lenght of correct answers:', t
print 'threshold = ', thresh

initial_corpus size: 344
babelnet results size of the topic "Biodiversity Conservation": 7481
ground truth size 55
intersection with the ground truth: 12
babel precision:  0.00160406362786
babel recall:  0.218181818182
initial_corpus size: 344
3sh tresh results of Biodiversity Conservation
precision: 0.00243013365735 recall: 0.0727272727273
number of selected items "3sh results lenght": 1646 lenght of correct answers: 55
threshold =  0.65


In [12]:
#Evaluate 3SH results at topN
def eval_at_n(results_file, topic, synset):
    babelnet_results, test = babelnet_syn_get_input(topic, synset)
    n = len(babelnet_results) 
    results = pickle.load(open(results_file, 'rb'))
    topn = pd.DataFrame(results.items()[:n])
    t = len(test)
    recall = len(find_intersection(test,topn[0]))/float(t)
    precision = len(find_intersection(test,topn[0]))/float(n)
    return precision, recall, n, t

In [40]:
synset = ['Substance abuse', 'drug abuse', 'habit', 'addiction', 'dependency', 'Abuse potential', 'Abusing drugs', 'Abusive drug use', 'Anti-drug', 'Cannabis abuse', 'Drug-abuse', 'Drug misuse', 'Drug prevention', 'Drugs of abuse', 'Illegal drug abuse', 'Illegal drug use', 'Misuse of drugs', 'Narcotic abuse theory', 'Nondependent abuse of drugs', 'Prescription drug abuse', 'Prescription Drug Misuse']
p , r, n, t = eval_at_n('results/Substance_results.pickle', 'Substance abuse', synset)
print 'precision:', p, 'recall:', r
print 'number of selected items "babelnet results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 2658
precision: 0.00810845052578 recall: 0.137339055794
number of selected items "babelnet results lenght": 7893 lenght of correct answers: 466


In [41]:
synset = ['Respiratory system', 'systema respiratorium', 'Respiration organ', 'Breathing organ', 'Breathing system', 'Development of respiratory system', 'Human Respiration', 'Human respiratory system', 'Pulmonary respiration', 'Pulmonary system', 'Respatory system', 'Respiration of human', 'Respiration system, Respiratory', 'Respiratory organs', 'Respiratory system agents', 'Respiratory system disorders', 'Respiratory systems', 'Respitory System', 'Subglottic airway', 'The respiratory system']
p , r, n, t = eval_at_n('results/Respiratory_system_results.pickle', 'Respiratory system', synset)
print 'precision:', p, 'recall:', r
print 'number of selected items "babelnet results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 795
precision: 0.00390625 recall: 0.021164021164
number of selected items "babelnet results lenght": 1024 lenght of correct answers: 189


In [42]:
synset = ['Remote sensing', 'Infrared remote sensing', 'Passive remote sensing', 'Remote-sensing', 'Remote-Sensing Image', 'Remote Sensing Satellites', 'Remote sensor']
topic = 'Remote sensing'
p , r, n, t = eval_at_n('results/Remote_sensing_results.pickle', topic, synset)
print 'precision:', p, 'recall:', r
print 'number of selected items "babelnet results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 4642
precision: 0.0345784418356 recall: 0.135906040268
number of selected items "babelnet results lenght": 4685 lenght of correct answers: 1192


In [43]:
synset = ['international relations', 'international affairs', 'world affairs', 'International politics', 'Intrel', 'Diplomacy and Statecraft', 'Diplomatic relationship', 'Global relations', 'International relation', 'International Relation Studies', 'International trade relations', 'Interstate relations', 'Levels of Analysis in international relations', 'Study of International Relations']
topic = 'international relations'
p , r, n, t = eval_at_n('results/international_relations_results.pickle', topic, synset)
print 'precision:', p, 'recall:', r
print 'number of selected items "babelnet results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 724
precision: 0.0030518819939 recall: 0.05
number of selected items "babelnet results lenght": 983 lenght of correct answers: 60


In [44]:
synset = ['Emergency medicine', 'Emergency care', 'Emergency med', 'Emergency treatment', 'Emergent condition', 'Emergentology', 'Er physician']
topic = 'Emergency medicine'
p , r, n, t = eval_at_n('results/Emergence_results.pickle', topic, synset)
print 'precision:', p, 'recall:', r
print 'number of selected items "babelnet results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 810
precision: 0.00290065264685 recall: 0.0140350877193
number of selected items "babelnet results lenght": 1379 lenght of correct answers: 285


In [45]:
synset = ['conservation biology', 'Animal conservation', 'Biological Conservation', 'Conservation', 'Biodiversity conservation', 'Conservation biologist', 'Conservation biologists', 'Conservation of natural resources', 'Conservation of wildlife', 'Conservation priority', 'Conservation science', 'Earth biologist', 'Earth biology', 'Ecological conservation', 'History of conservation biology', 'Wildlife Conservation']
topic = 'Biodiversity Conservation'
p , r, n, t = eval_at_n('results/Biodiversity_Conservation_results.pickle', topic, synset)
print 'precision:', p, 'recall:', r
print 'number of selected items "babelnet results lenght":', n, 'lenght of correct answers:', t

initial_corpus size: 344
precision: 0.0022724234728 recall: 0.309090909091
number of selected items "babelnet results lenght": 7481 lenght of correct answers: 55


In [38]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000:2000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000:3000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000:4000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000:5000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[5000:6000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[6000:7000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[7000:8000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[8000:9000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[9000:10000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[10000:20000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[20000:30000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[30000:40000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[40000:50000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[50000:60000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[60000:70000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[70000:80000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[80000:90000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[90000:100000])[0], babelnet_results))

304
213
169
163
137
96
98
103
103
59
531
321
220
190
136
133
130
102
100


In [42]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[100000:200000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[200000:300000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[300000:400000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[400000:500000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[500000:600000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[600000:700000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[700000:800000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[800000:900000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[900000:1000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000000:2000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000000:3000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000000:4000000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000000:])[0], babelnet_results))

649
272
221
173
179
182
181
151
177
1356
852
506
45


In [39]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:2000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:3000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:4000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:5000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:6000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:7000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:8000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:9000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:10000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:20000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:30000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:40000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:50000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:60000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:70000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:80000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:90000])[0], babelnet_results))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:100000])[0], babelnet_results))

304
517
686
849
986
1082
1180
1283
1386
1445
1976
2297
2517
2707
2843
2976
3106
3208
3308


In [40]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000:2000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000:3000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000:4000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000:5000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[5000:6000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[6000:7000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[7000:8000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[8000:9000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[9000:10000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[10000:20000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[20000:30000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[30000:40000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[40000:50000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[50000:60000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[60000:70000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[70000:80000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[80000:90000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[90000:100000])[0], babel_test_intersection))

1
1
1
0
0
0
1
0
0
1
1
3
2
0
0
0
0
0
0


In [43]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[100000:200000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[200000:300000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[300000:400000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[400000:500000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[500000:600000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[600000:700000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[700000:800000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[800000:900000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[900000:1000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[1000000:2000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[2000000:3000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[3000000:4000000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[4000000:])[0], babel_test_intersection))

0
1
0
0
0
0
0
0
0
0
0
0
0


In [41]:
print len (find_intersection(pd.DataFrame(s3h_results.items()[:1000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:2000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:3000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:4000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:5000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:6000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:7000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:8000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:9000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:10000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:20000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:30000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:40000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:50000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:60000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:70000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:80000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:90000])[0], babel_test_intersection))
print len (find_intersection(pd.DataFrame(s3h_results.items()[:100000])[0], babel_test_intersection))

1
2
3
3
3
3
4
4
4
5
6
9
11
11
11
11
11
11
11


In [27]:
pd.DataFrame(s3h_results.items()[:1000])

,0,1
0,0A8071F67FEE1A6BF14E5069288C9E67B8A3F2AA,1.000
1,2946E8F7A320DEF1BEFBB26A21CB95E0492FE580,1.000
2,4B1F119CA2211850D0FA755EB5D43D0C26A0358B,1.000
3,396205226F7DDC8B48E8FBEF16684C7703D905D1,1.000
4,FA9D1081F58F45F8168813C84C6D9223BBB1873E,1.000
5,7E4EF8002D4FEA0DE518902EC3D79AE81B29D79A,1.000
6,277C9AC0AF7CB753DBF8609EE57C946AAEF0A172,1.000
7,FAD58615F61C6D0F70C3F054B27D8D7B75AC7C34,1.000
8,D037DE25D3FAE7E3D1F7EC618602687C735224C8,1.000
9,71ADE7014F39A00FDE2D4530BCDD5DF7E509B179,1.000


In [52]:
def eval_babelnet(results, top_n, test):
	top = results[:top_n]
	print 'matched rate at '+str(top_n)+':' , len(find_intersection(test,top))/float(len(test))

In [22]:
print 'BabelNet Method Evaluation:'
for i in range(1000,11000,1000):
    eval_babelnet(babelnet_results, i, test)

BabelNet Method Evaluation:
matched rate at 1000: 0.0181818181818
matched rate at 2000: 0.0181818181818
matched rate at 3000: 0.0363636363636
matched rate at 4000: 0.0727272727273
matched rate at 5000: 0.109090909091
matched rate at 6000: 0.109090909091
matched rate at 7000: 0.145454545455
matched rate at 8000: 0.218181818182
matched rate at 9000: 0.218181818182
matched rate at 10000: 0.218181818182


In [47]:
def eval_use_case(sorted_results, top_n, test):
	top = pd.DataFrame(sorted_results.items()[:top_n])
	print 'matched rate at '+str(top_n)+':' , len(find_intersection(test,top[0]))/float(len(test))

In [18]:
print 'Evaluation:'
for i in range(1000,11000,1000):
    eval_use_case(s3h_results, i, test)

Evaluation:
matched rate at 1000: 0.127272727273
matched rate at 2000: 0.236363636364
matched rate at 3000: 0.290909090909
matched rate at 4000: 0.309090909091
matched rate at 5000: 0.327272727273
matched rate at 6000: 0.345454545455
matched rate at 7000: 0.363636363636
matched rate at 8000: 0.436363636364
matched rate at 9000: 0.436363636364
matched rate at 10000: 0.454545454545


In [45]:
print 'Evaluation Hyprid:'
for i in range(1000,11000,1000):
    eval_use_case(s3h_syn_results, i, test)

Evaluation Hyprid:
matched rate at 1000: 0.109090909091
matched rate at 2000: 0.218181818182
matched rate at 3000: 0.236363636364
matched rate at 4000: 0.327272727273
matched rate at 5000: 0.363636363636
matched rate at 6000: 0.381818181818
matched rate at 7000: 0.381818181818
matched rate at 8000: 0.418181818182
matched rate at 9000: 0.454545454545
matched rate at 10000: 0.472727272727


In [57]:
mlt_results = pickle.load(open('elasticsearch/mlt_initial_corpus_results.pickle', 'rb'))

In [53]:
print 'MLT Method Evaluation:'
for i in range(1000,11000,1000):
    eval_babelnet(mlt_results, i, test)

MLT Method Evaluation:
matched rate at 1000: 0.0
matched rate at 2000: 0.0
matched rate at 3000: 0.0
matched rate at 4000: 0.0
matched rate at 5000: 0.0
matched rate at 6000: 0.0
matched rate at 7000: 0.0
matched rate at 8000: 0.0
matched rate at 9000: 0.0
matched rate at 10000: 0.0


In [59]:
find_intersection(mlt_results, test)

[]

In [61]:
len(test)

55

In [60]:
mlt_results

[u'205E7834346C3A1A1192C11E52B56AA5ACD08D00',
 u'C57EE34326E4C78F8A45F036F2D29F6F9E0A40B4',
 u'50DCDEA86FBA19D63FAC7CED48FFE844FEE3D5DA',
 u'6F1C1FC32134D9EB42F204EA54EFF076B0B84276',
 u'FF0EBD3588C293F6D2423BE2F1C6A33826919DE2']

In [100]:
area = 'Area studies'
synset = area.split(',')
topic = 'Area Studies'
babelnet_eval(topic, synset)

initial_corpus size: 83
results size of the topic "Area Studies": 83
ground truth size 2
intersection with the ground truth: 0


In [101]:
bab = 'behavioural sciences, Behavioral science, Behavioral sciences, Behavioral disciplines, Behavioral scientist, Behavioral scientists, Behavioural disciplines, Behavioural science, Behavioural scientist'
synset = bab.split(',')
topic = 'Behavioral Sciences'
babelnet_eval(topic, synset)

initial_corpus size: 121
results size of the topic "Behavioral Sciences": 357
ground truth size 21
intersection with the ground truth: 4


In [102]:
bab = 'biophysics, Biological physics, Biophysical, Biophysicists, History of biophysics'
synset = bab.split(',')
topic = 'biophysics'
babelnet_eval(topic, synset)

initial_corpus size: 275
results size of the topic "biophysics": 3674
ground truth size 323
intersection with the ground truth: 19


In [103]:
bab = 'artificial intelligence, AI, artilect, Artifical intelligence, Cognitive systems, Digital being, Machine intelligence, A.I., A I, AI-D, AI ethics, AI implications, AI Robotics, AI scripting, Aretificial intelligence, Artificial-intelligence, Artificial conciousness, Artificial inteligence, Artificial intellect, Artificial intellegence, Artificial Intelligence., Artificial intelligence for development, Artificial Intelligence Program, Artificial intelligences, Artificially-intelligent, Artificially intelligent, Artificual intelligence, Cognitive simulation, Cognitive system, Commonsense AI, Computational Rationality, Computer AI, Intelligent machine, Machine thought, Machine understanding, Ontology based approach, Pseudo intelligence, Semi-AI, Semi AI, Simulated intelligence, Soft AI, Sub-symbolic, Subsymbolic, The Artificial Intelligence, The Theory of Artificial Intelligence'
synset = bab.split(',')
topic = 'artificial intelligence'
babelnet_eval(topic, synset)

initial_corpus size: 2091
results size of the topic "artificial intelligence": 7903
ground truth size 657
intersection with the ground truth: 43


In [104]:
bab = 'cybernetics, Cybernetic, Cybernetic system, cybernetician, Cyberneticians, cyberneticist, Tha Masta'
synset = bab.split(',')
topic = 'Cybernetics'
babelnet_eval(topic, synset)

initial_corpus size: 291
results size of the topic "Cybernetics": 511
ground truth size 596
intersection with the ground truth: 47


In [105]:
bab = 'information system, data system, information systems, system info, Business computing, Computer information system, Computer information systems, Elements of Information System, Information in Computer Science, Information systems and technology, Information systems discipline, Information systems theory, Informationssystem'
synset = bab.split(',')
topic = 'Information Systems'
babelnet_eval(topic, synset)

initial_corpus size: 5033
results size of the topic "Information Systems": 8440
ground truth size 1313
intersection with the ground truth: 298


In [106]:
bab = 'software engineering, Software Hut, Characteristics of software engineering, Computer Software Engineering, In-House Design, Software eng, software engineer, Software engineers, Software management, Software Specialist'
synset = bab.split(',')
topic = 'Software Engineering'
babelnet_eval(topic, synset)

initial_corpus size: 928
results size of the topic "Software Engineering": 1058
ground truth size 241
intersection with the ground truth: 5


In [107]:
bab = 'aerospace, Aeronautical-Astronautical, Aerospace projects, Aerospace Revolution, Ærospace'
synset = bab.split(',')
topic = 'Aerospace'
babelnet_eval(topic, synset)

initial_corpus size: 1560
results size of the topic "Aerospace": 1560
ground truth size 122
intersection with the ground truth: 0


In [108]:
bab = 'immunology, immunobiology, immunological, immunologist, Clinical immunology, Classical immunology, Evolutionary immunology, Immunologic, Immunologists, Imunologys'
synset = bab.split(',')
topic = 'Immunology'
babelnet_eval(topic, synset)

initial_corpus size: 1514
results size of the topic "Immunology": 8787
ground truth size 4769
intersection with the ground truth: 203


In [109]:
bab = 'infection, infectious disease, communicable diseases, contagion, Infectious diseases, Acute infection, AIDS-related bacterial infections, AIDS-related viral infections, Anti-infective, Anti-infectives, Antiinfective, Bacterial Infections, Communicable disease, Contagious diseases, Definition to contagious, Infect, Infecting, Infectiology, Infections, Infectious, Infectious disease epidemiology, Infectious disease medicine, Infectology, Local infection, Primary infection, Rochalimea infections, Secondary infection, Tropical bacterial infections, Tropical infections, Viral Infections, Wound colonization, Wound infection'
synset = bab.split(',')
topic = 'Infectious Diseases'
babelnet_eval(topic, synset)

initial_corpus size: 2588
results size of the topic "Infectious Diseases": 8343
ground truth size 1375
intersection with the ground truth: 50


In [110]:
bab = 'limnology, lakelore, Freshwater science, History of limnology, Limnographer, Limnological'
synset = bab.split(',')
topic = 'Limnology'
babelnet_eval(topic, synset)

initial_corpus size: 108
results size of the topic "Limnology": 394
ground truth size 73
intersection with the ground truth: 16


In [111]:
bab = 'linguistics, Linguists, Language specialist, Language studies, Lingistics, Linguism, linguist, Linguistic, Linguistic layer, Linguistic layers, Linguistic science, Linguistic scientist, Linguistical, Linguistically, Linguistics and Languages, Linguistics layer, Linguistics layers, Linguistsics, Linguithtics, Lingustics, Speech scientist, Speech scientists, Statistical linguistics, Study of language, Verbal communication'
synset = bab.split(',')
topic = 'Linguistics'
babelnet_eval(topic, synset)

initial_corpus size: 1050
results size of the topic "Linguistics": 6898
ground truth size 214
intersection with the ground truth: 61


In [112]:
bab = 'literature, Literary art, Literary, Literary arts, Literary work, Literary works, LiteraryArt, Literature by region, Literatures, Litterature, Ltierature, Prose fiction'
synset = bab.split(',')
topic = 'Literature'
babelnet_eval(topic, synset)

initial_corpus size: 7941
results size of the topic "Literature": 7357
ground truth size 860
intersection with the ground truth: 107


In [113]:
bab = 'Biomaterial, Biomaterials Engineering, Bio material, Biomaterials'
synset = bab.split(',')
topic = 'Biomaterials'
babelnet_eval(topic, synset)

initial_corpus size: 2442
results size of the topic "Biomaterials": 3649
ground truth size 1020
intersection with the ground truth: 92


In [114]:
bab = 'ceramics, Ceramic art, Art pottery, Art ware, Ceramic artist, Ceramic paint, Ceramics art, Fine art pot, Vase painting'
synset = bab.split(',')
topic = 'Ceramics'
babelnet_eval(topic, synset)

initial_corpus size: 8256
results size of the topic "Ceramics": 8249
ground truth size 1276
intersection with the ground truth: 0


In [115]:
bab = 'textile, cloth, fabric, material, Cloths, Fabrical, Fabricalities, Fabricality, Fabrically, Fabrics, Interlock cloth, Textile fibre, Textiled, Textiler, Textilers, Textiles, Textiling, Tray-cloth'
synset = bab.split(',')
topic = 'Textiles'
babelnet_eval(topic, synset)

initial_corpus size: 765
results size of the topic "Textiles": 7217
ground truth size 158
intersection with the ground truth: 97


In [116]:
bab = ' mechanics, History of mechanics, Mechanical processes, Particle mechanics, Theoretical mechanics'
synset = bab.split(',')
topic = 'Mechanics'
babelnet_eval(topic, synset)

initial_corpus size: 8210
results size of the topic "Mechanics": 8222
ground truth size 4640
intersection with the ground truth: 1


In [118]:
bab = 'medical ethics, Health ethics, Ethics in medicine, Medical ethicist'
synset = bab.split(',')
topic = 'Medical Ethics'
babelnet_eval(topic, synset)

initial_corpus size: 382
results size of the topic "Medical Ethics": 413
ground truth size 174
intersection with the ground truth: 0


In [119]:
bab = 'Health informatics, Biomedical informatics, Clinical Informatics, Clinical Research Informatics, Gero-Informatics, Health Informatics Law, Medical informatics, Nursing informatics, Clinical informatist, Consumer Health Informatics, Dental informatics, Geroinformatics, Health care informatics, Health information, Health information system, Health information systems, Health management information system, Healthcare, Information Technology, Healthcare informatics, Healthcare information system, Healthcare Information Technology, Heath informatics, Medical computer science, Pharmacy informatics, Veterinary informatics'
synset = bab.split(',')
topic = 'Medical Informatics'
babelnet_eval(topic, synset)

initial_corpus size: 139
results size of the topic "Medical Informatics": 7437
ground truth size 94
intersection with the ground truth: 20


In [120]:
bab = 'microbiology, Rapid microbiology, History of microbiology, Microbiological, Microbiological techniques'
synset = bab.split(',')
topic = 'Microbiology'
babelnet_eval(topic, synset)

initial_corpus size: 1603
results size of the topic "Microbiology": 5906
ground truth size 1320
intersection with the ground truth: 147


In [121]:
bab = 'microscopy, 3D-SIM-microscopy, Amateur microscopy, Bioimaging, Infrared microscopy, IR microscopy, Laser microscopy, Light microscopy, Microscopic examination, Microscopically, Microscopist, Oblique illumination, Polarized light microscope'
synset = bab.split(',')
topic = 'Microscopy'
babelnet_eval(topic, synset)

initial_corpus size: 8418
results size of the topic "Microscopy": 8547
ground truth size 6819
intersection with the ground truth: 253


In [122]:
bab = 'mineralogy, Mineralogical, Mineralology, Minerology, Quantum mineralogy'
synset = bab.split(',')
topic = 'Mineralogy'
babelnet_eval(topic, synset)

initial_corpus size: 2407
results size of the topic "Mineralogy": 4324
ground truth size 196
intersection with the ground truth: 48


In [123]:
bab = "music, music lover, Musical interpretation, 1st Art, Auditory art, Chord, Chromatic, chromatic scale, Chromaticism, composer, consonance, Consonance and dissonance, Dissonance, Flashygoodness, Interpretation, Meaning, Mucic, Music's, Music And Musical Instruments, Musica instrumentalis, Musical, musician, Musicians, Musicologist, Muzic, pianist, Romantic music, Singer-songwriter, songwriter, symphony"
topic = 'Music'
babelnet_eval(topic, synset)

initial_corpus size: 2449
results size of the topic "Music": 4324
ground truth size 150
intersection with the ground truth: 0


In [124]:
bab = 'mycology, fungology, History of mycology, Micology, Mycological, Mycologists, Study of fungi'
synset = bab.split(',')
topic = 'Mycology'
babelnet_eval(topic, synset)

initial_corpus size: 179
results size of the topic "Mycology": 542
ground truth size 530
intersection with the ground truth: 3


In [125]:
bab = 'neuroimaging, Brain imaging, Brain scanning, Brain function map, Brain scan, Brain scans, Functional neurological mapping, Neuro-imaging, Neuroradiography'
synset = bab.split(',')
topic = 'Neuroimaging'
babelnet_eval(topic, synset)

initial_corpus size: 2497
results size of the topic "Neuroimaging": 3679
ground truth size 522
intersection with the ground truth: 52


In [126]:
bab = 'neuroscience, Neuroscience studies, History of behavioral neuroscience, Interface of neuroscience with all liberal arts disciplines, Neural science, Neurobiological, Neuroscience and popular culture, Neuroscience and society, Neuroscience and the media, Neuroscience education: undergraduate models, best practices, Neurosciences, Neuroscientific, neuroscientist, Neuroscince'
synset = bab.split(',')
topic = 'Neurosciences'
babelnet_eval(topic, synset)

initial_corpus size: 202
results size of the topic "Neurosciences": 4260
ground truth size 95
intersection with the ground truth: 3


In [127]:
bab = 'nursing, Nursing Science, Staff nurse, Adult nursing, Flightnurse, nurse, Nursing History, Nursing Officer, Nursing practice, Nursing skills, Nursing staff, Nursing Student, Nursing unit, Nurxing, Practice of nursing'
synset = bab.split(',')
topic = 'Nursing'
babelnet_eval(topic, synset)

initial_corpus size: 8160
results size of the topic "Nursing": 8252
ground truth size 3282
intersection with the ground truth: 276


In [128]:
bab = 'oceanography, oceanology, Marine science, Marine Sciences, History of oceanography, Maps of the Oceans, Marine research, Marine scientist, Ocean geography, Ocean science, Ocean sciences, Oceanographic, Oceanologia, Oceanologist, Oceans and Oceanography, Undersea Exploration, Underwater exploration'
synset = bab.split(',')
topic = 'Oceanography'
babelnet_eval(topic, synset)

initial_corpus size: 523
results size of the topic "Oceanography": 3231
ground truth size 210
intersection with the ground truth: 52


In [129]:
bab = 'oncology, oncologist, AllergoOncology, Cancer care, Medical oncology, Oncologic, Oncological, Oncologists, Oncology research, Oncology unit, Pediatric cancers'
synset = bab.split(',')
topic = 'Oncology'
babelnet_eval(topic, synset)

initial_corpus size: 3922
results size of the topic "Oncology": 5705
ground truth size 2937
intersection with the ground truth: 107


In [130]:
bab = 'ophthalmology, All India Ophthalmological Conference, Clincial ophthalmology, Clincial opthalmology, Clinical ophtalmology, Clinical Ophthalmology, Clinical Opthalmology, General ophthalmic services, Oculists, Oftamology, Ofthamology, Ophthalmic surgeon, Ophthalmologic, Ophthalmological, Ophthalmologicals, Ophthalmologists, Ophthamologist, Ophthamology, Optamology, Opthalmological, Opthalmologist, Opthalmology, Opthamologist, Opthamology, Opthomologist, Society for clincial ophthalmology, Society for clincial opthalmology, Society for clinical ophthalmology, Society for clinical opthalmology, Vision care'
synset = bab.split(',')
topic = 'Ophthalmology'
babelnet_eval(topic, synset)

initial_corpus size: 958
results size of the topic "Ophthalmology": 2096
ground truth size 459
intersection with the ground truth: 25


In [131]:
bab = 'optics, Classical optics, Light physics, Optical, Optical system'
synset = bab.split(',')
topic = 'Optics'
babelnet_eval(topic, synset)

initial_corpus size: 5349
results size of the topic "Optics": 8730
ground truth size 3286
intersection with the ground truth: 271


In [132]:
bab = 'ornithology, birdlore, Bird watching towers, History of ornithology, Ornathology, Ornithologic, Ornithological, Ornithologically, Ornothology, Orthinology, Science of birds, Study of Birds'
synset = bab.split(',')
topic = 'Ornithology'
babelnet_eval(topic, synset)

initial_corpus size: 28
results size of the topic "Ornithology": 69
ground truth size 6
intersection with the ground truth: 3


In [133]:
bab = 'orthopedics, orthopaedics, Orthopedic surgery, Orthopaedic surgery, Orthopedic surgeon, Bone doctor, Bone surgeon, Bone surgery, Bonemender, Orthepedic surgeon, Orthopaedic, Orthopaedic procedure, Orthopaedic surgeon, Orthopaedists, Orthopaedy, Orthopedic, Orthopedic medicine, Orthopedic procedure, Orthopedic procedures, Orthopedic surgeons, Orthopedy, Orthopod, Orthopods, Orthopædic, Orthopædics'
synset = bab.split(',')
topic = 'Orthopedics'
babelnet_eval(topic, synset)

initial_corpus size: 178
results size of the topic "Orthopedics": 5949
ground truth size 51
intersection with the ground truth: 19


In [136]:
bab = 'otorhinolaryngology, otolaryngology, rhinolaryngology, ear, nose and throat, ENT doctor, Aural surgeon, E.N.T., Ear, nose, and throat, Ear, nose, and throat specialist, Ear, nose and throat specialist, Ear nose and throat, Ear surgeon, Ear surgery, Ears, nose, mouth, and throat, Ears, Nose and Throat, ENT medicine, ENT specialist, ENT surgery, ENTs, Oto-rhino-laryngology, Oto rhino laryngology, Otolaryngologists, Otolaryngology- head and neck surgery, Otolaryngology - Head  Neck Surgery, Otolaryngology - head and neck surgery, Otolarynology, Otoretolaryngologyst, Otorhinolaryngologic, Otorhinolaryngologic diseases, Otorhinolaryngologic neoplasms, Otorhinolaryngologic surgical procedures, Otorhinolaryngological, Otorhinolaryngologists, Otorhinolaryngology, head and neck surgery, Otorhynolaryngology'
synset = bab.split(',')
topic = 'Otorhinolaryngology'
babelnet_eval(topic, synset)

initial_corpus size: 59
results size of the topic "Otorhinolaryngology": 8382
ground truth size 6
intersection with the ground truth: 3


In [137]:
bab = 'paleontology, fossilology, palaeontology, paleontologist, Dinosaur palaeontology, Doctor of Paleontology, Palaentologist, Palaentology, Palaeontological, Palaeontologies, Palaeontologist, Palaeontologists, Palaeoology, Palaeoooelogy, Palaeooology, Palaeooölogy, Paleantologist, Paleantology, Palentoligsts, Palentologists, Palentology, Paleonthologist, Paleontological, Paleontologists, Paleoology, Palæontologist, Palæontology'
synset = bab.split(',')
topic = 'Paleontology'
babelnet_eval(topic, synset)

initial_corpus size: 99
results size of the topic "Paleontology": 824
ground truth size 63
intersection with the ground truth: 8


In [138]:
# left becasue of small testset
bab = 'parasitology, parasitologist, Parasite Ecology, History of parasitology, Medical parasitology, Parasitologically'
synset = bab.split(',')
topic = 'Parasitology'
babelnet_eval(topic, synset)

initial_corpus size: 199
results size of the topic "Parasitology": 326
ground truth size 365
intersection with the ground truth: 1


In [139]:
bab = 'pathology, Pathology as a medical specialty, General pathology, Autopsy Surgeon, Pathoanatomy, Pathobiology, Pathologic processes, Pathological, Pathological case, Pathologically, Pathologies, Pathologism, Pathologisms, Pathology as a science, Study of disease'
synset = bab.split(',')
topic = 'Pathology'
babelnet_eval(topic, synset)

initial_corpus size: 8384
results size of the topic "Pathology": 8544
ground truth size 2726
intersection with the ground truth: 174


In [140]:
bab = 'pediatrics, paediatrics, pediatric medicine, pedology, pediatrician, paediatrician, pædiatrics, Paediatric, Paediatricians, Pediatric, Pediatricians, Pediatrist, Pediatry, Pædiatric, Pædiatrician'
synset = bab.split(',')
topic = 'Pediatrics'
babelnet_eval(topic, synset)

initial_corpus size: 565
results size of the topic "Pediatrics": 8233
ground truth size 747
intersection with the ground truth: 223


In [141]:
bab = 'Peripheral vascular disease, Arterial occlusive disease, Fontaine classification, Fontaine stage, PAOD, Peripheral arterial disease, Peripheral arterial occlusive disease, Peripheral artery disease, Peripheral artery occlusive disease, Peripheral vascular diseases'
synset = bab.split(',')
topic = 'Peripheral Vascular Disease'
babelnet_eval(topic, synset)

initial_corpus size: 863
results size of the topic "Peripheral Vascular Disease": 1540
ground truth size 164
intersection with the ground truth: 23


In [142]:
bab = 'philosophy, Philosophic, Philosophical, Philosophical Subdisciplines, Philosophically, Philosophized, Applied philosophy, Branch of philosophy, Branches of philosophy, DefinitionOfPhilosophy, Definitions of philosophy, Filosofy, Philisophical, Philisophy, Philo-sophy, Philosophae, philosopher, Philosophers, Philosophhy, Philosophiae, PhilosophicalSubdisciplines, Philosophicians, Philosophies, Philosophise, Philosophised, Philosophiser, Philosophisers, Philosophises, Philosophising, Philosophize, Philosophizer, Philosophizers, Philosophizes, Philosophizing, PhilosophyAndLogic, Philosphical, Philosphy, Phylosophy, Roman ideals, Sage'
synset = bab.split(',')
topic = 'Philosophy'
babelnet_eval(topic, synset)

initial_corpus size: 6692
results size of the topic "Philosophy": 7116
ground truth size 677
intersection with the ground truth: 110


In [143]:
bab = 'Condensed matter physics, Condensed-matter physics, Condensed matter, Bulk matter, Condenced matter, Condensed matter physicist, Condensed matter system, Condensed matter theory, Condensed phase, History of condensed matter physics, Physics of condensed matter, Theoretical condensed matter physics'
synset = bab.split(',')
topic = 'Condensed Matter'
babelnet_eval(topic, synset)

initial_corpus size: 581
results size of the topic "Condensed Matter": 1523
ground truth size 1514
intersection with the ground truth: 1


In [25]:
bab = 'Condensed matter physics, Condensed-matter physics, Condensed matter, Bulk matter, Condenced matter, Condensed matter physicist, Condensed matter system, Condensed matter theory, Condensed phase, History of condensed matter physics, Physics of condensed matter, Theoretical condensed matter physics'
synset = bab.split(',')
topic = 'Condensed Matter'
res_pickle = "results/res__Condensed_matter"
eval_all_at_thresh_lst(res_pickle, topic, synset, thresh=0.75)


initial_corpus size: 581
babelnet results size of the topic "Condensed Matter": 1523
ground truth size 1514
intersection with the ground truth: 1
babel precision:  0.000656598818122
babel recall:  0.000660501981506
initial_corpus size: 581


(0.00881057268722467, 0.05812417437252312, 9988, 1514, 0.75)

In [144]:
bab = 'physiology, animal physiology, History of physiology, Institutes of Medicine, Phisiology, Physiologic, Physiological, Physiologists'
synset = bab.split(',')
topic = 'Physiology'
babelnet_eval(topic, synset)

initial_corpus size: 8525
results size of the topic "Physiology": 8494
ground truth size 2761
intersection with the ground truth: 123


In [145]:
bab = 'psychiatry, psychopathology, psychological medicine, Adult Psychiatry, Criticism of psychiatry, Ethics of psychiatry, History of psychiatry, Mental Pathology, Modern psychiatry, Personalistic disease theories, Psichiatry, Psychaitry, Psychiatric, Psychiatric medicine, Psychiatric syndrome, Psychiatric treatment, psychiatrist, Psychiatrists, Psychological pathology, Psychopathological, Psychopathologist, Psycopathological, Psycopathology, Pyschiatric care, Pyschiatry'
synset = bab.split(',')
topic = 'Psychiatry'
babelnet_eval(topic, synset)

initial_corpus size: 2145
results size of the topic "Psychiatry": 7448
ground truth size 1700
intersection with the ground truth: 250


In [146]:
bab = 'psychology, psychological science, Human psychology, Psychological, Psychologically, Criticism of psychology, Human trait, Hyde event, Phsycology, Physcology, Professional psychology, Psychogenics, Psychologic, Psychological sciences, Psychological terms, Psychological theories, Psychological theory, psychologist, Psychologists, Psychology/rewrite, Psycologic, Psycological, Psycology, Pyhscology, Self-report study, WEIRD'
synset = bab.split(',')
topic = 'Psychology'
babelnet_eval(topic, synset)

initial_corpus size: 6857
results size of the topic "Psychology": 7187
ground truth size 1871
intersection with the ground truth: 134


In [147]:
bab = 'psychoanalysis, depth psychology, analysis, Psychoanalyst, Psychoanalysts, Analysand, Criticism of Freud, Criticism of Sigmund Freud, Depth psychologist, Depth therapy, Freudian analysis, Freudian analyst, Freudian psychology, History of psychoanalysis, Jungian interpreretation of religion, Psychanalysis, Psycho-analysis, Psycho Analysis, Psychoanalitic theory, Psychoanalysis/Archives/2003-2005, Psychoanalytic, Psychoanalytic perspective, Psychoanalytic Psychiatry, Psychoanalytic school, Psychoanalytic therapy, PSYCHOANALYTIC TREATMENT, Psychoanalytical, Psychoanalytical feminism, Psychoanalytical theory, Psychoanalyze, Psychoanaylsis, Psychoannalysis, Psychoanylists, Psychological analysis, Psyco-analysis, Psycoanalysis'
synset = bab.split(',')
topic = 'Psychoanalysis'
babelnet_eval(topic, synset)

initial_corpus size: 733
results size of the topic "Psychoanalysis": 7176
ground truth size 129
intersection with the ground truth: 59


In [148]:
bab = 'rehabilitation, Neurocognitive Rehabilitation, Neurological rehabilitation, Neuropsychological rehabilitation, Rehabilitation Neuropsychology'
synset = bab.split(',')
topic = 'Rehabilitation'
babelnet_eval(topic, synset)

initial_corpus size: 7458
results size of the topic "Rehabilitation": 7449
ground truth size 773
intersection with the ground truth: 0


In [149]:
bab = 'aith, religion, religious belief, belief, creed, Faiths, Religions, Religious beliefs, Allegory of faith, Co-religionism, Co-religionist, Co-religionists, Coreligionism, Coreligionist, Coreligionists, Creating Stories, Dereligionization, Faithful, Faithfully, Fictitous, Magical thinking/Revised, Magickal thinking, Relegious, Relgion, Relig, Relig., Religionistic, Religionistical, Religionistically, Religionists, Religious concepts, Religious faith, Religious issues, Religious tradition, Religious traditions, Religiousity, Religon, Relligion, Totalitarian religious group'
synset = bab.split(',')
topic = 'Religion'
babelnet_eval(topic, synset)

initial_corpus size: 4073
results size of the topic "Religion": 7116
ground truth size 587
intersection with the ground truth: 84


In [150]:
bab = 'rheumatology, Rheumatic, Rheumatologic, Rheumatologists'
synset = bab.split(',')
topic = 'Rheumatology'
babelnet_eval(topic, synset)

initial_corpus size: 837
results size of the topic "Rheumatology": 2780
ground truth size 180
intersection with the ground truth: 27


In [151]:
bab = 'robotics, Autonomous Systems, Flying robots, Robotic, Robotic leg, Robotic legs, Socionics, Under water robotics, Unmanned systems'
synset = bab.split(',')
topic = 'Robotics'
babelnet_eval(topic, synset)

initial_corpus size: 1231
results size of the topic "Robotics": 3705
ground truth size 747
intersection with the ground truth: 242


In [152]:
bab = 'sociology, Sociological, Sociologists, Marketing sociologist, Marketing sociology, Scientific sociology, Social physics, Socialogy, Sociol, Sociological inquiry, Sociological term, Sociological terms, sociologist, Sociology versus social theory, Sociology vs. Social Theory, Sosiology, Study of culture'
synset = bab.split(',')
topic = 'Sociology'
babelnet_eval(topic, synset)

initial_corpus size: 2356
results size of the topic "Sociology": 4718
ground truth size 698
intersection with the ground truth: 53


In [153]:
bab = 'soil science, Soil sciences, Soil scientist'
synset = bab.split(',')
topic = 'soil science'
babelnet_eval(topic, synset)

initial_corpus size: 136
results size of the topic "soil science": 145
ground truth size 10
intersection with the ground truth: 0


In [154]:
bab = 'spectroscopy, spectrographic analysis, spectrometry, spectroscopic analysis, spectrum analysis, Fingerprint region, Laser spectroscopy, Optical spectroscopy, Atomic line, Atomic line spectra, Atomic spectra, Atomic spectral line, Electromagnetic spectroscopy, Emission spectrochemical analysis, Spectrochemical Analysis, Spectrography, Spectrology, Spectroscopic, Spectroscopist, Spectroscopists'
synset = bab.split(',')
topic = 'Spectroscopy'
babelnet_eval(topic, synset)

initial_corpus size: 8607
results size of the topic "Spectroscopy": 8513
ground truth size 7294
intersection with the ground truth: 287


In [155]:
bab = ' surgery, operation, surgical operation, surgical procedure, surgical process, Chirurgery, Chirurgical, Chirurgy, Complications of surgery, Corrective surgery, Elliptical excision, Emergency Surgery, Post-operation, Post-operative, Specialties in surgery, Sterile drapes, surgeon, Surgeons, Surgeries, Surgery in general practice, Surgery operation, Surgery specialties, Surgical, Surgical excision, Surgical excision of malignant lesions, Surgical specialties, Surgical technique, Surgically'
synset = bab.split(',')
topic = 'Surgery'
babelnet_eval(topic, synset)

initial_corpus size: 8455
results size of the topic "Surgery": 8271
ground truth size 6412
intersection with the ground truth: 565


In [156]:
bab = 'telecommunication, telecom, telecommunications, Communication systems, Communications in the Caribbean, Datacommunications, Electronic and Communication, Electronic communication, Electronic communications, Electronics and Communication, Electronics and Communications, General Telecommunications, Global telecommunications infrastructure, Mobile access, Telecomm, Telecomms, Telecommunication industry, Telecommunication Systems, Telecommunication technology, Telecommunications in the Caribbean, Telecommunications industry, Telecommunications systems, Telecommunications Technology, Telecommunnication, Telecoms, Telecomunication, Telecomunications, Telecomunnication, Transportation of information, Web communication'
synset = bab.split(',')
topic = 'Telecommunications'
babelnet_eval(topic, synset)

initial_corpus size: 2755
results size of the topic "Telecommunications": 5693
ground truth size 259
intersection with the ground truth: 69


In [158]:
bab = "dramaturgy, theater, theatre, dramatic art, dramatics, Children's theater, Theater art, Theater Arts, Theatre art, Theatre arts, A Theater, Children's theatre, Dramaturgic, Entertainment venue, Live theater, Live theatre, Stage productions, Theaters, Theatre companies, Theatre company, Theatre Studies, Theatrer, Theatres, Theatrical company, Theatrical scene, Theatrics"
synset = bab.split(',')
topic = 'Theater'
babelnet_eval(topic, synset)

initial_corpus size: 321
results size of the topic "Theater": 1312
ground truth size 12
intersection with the ground truth: 2


In [159]:
bab = 'thermodynamics, Thermo-dynamics, Thermodynamic functions, Applied thermodynamics, Classical thermodynamics, entropy, Macroscopic thermodynamics, Phenomenological thermodynamics, second law of thermodynamics, Termodynamics, Thermal behavior, Thermics, Thermodymanics, Thermodynamic, Thermodynamic function, Thermodynamic law, Thermodynamic Laws'
synset = bab.split(',')
topic = 'Thermodynamics'
babelnet_eval(topic, synset)


initial_corpus size: 6351
results size of the topic "Thermodynamics": 8375
ground truth size 2727
intersection with the ground truth: 392


In [28]:
bab = 'thermodynamics, Thermo-dynamics, Thermodynamic functions, Applied thermodynamics, Classical thermodynamics, entropy, Macroscopic thermodynamics, Phenomenological thermodynamics, second law of thermodynamics, Termodynamics, Thermal behavior, Thermics, Thermodymanics, Thermodynamic, Thermodynamic function, Thermodynamic law, Thermodynamic Laws'
synset = bab.split(',')
topic = 'Thermodynamics'
res_pickleckleickle = "results/res__Thermodynamics"
eval_all_at_thresh_lst(res_pickle, topic, synset, thresh=0.75)

initial_corpus size: 6351
babelnet results size of the topic "Thermodynamics": 8375
ground truth size 2727
intersection with the ground truth: 392
babel precision:  0.0468059701493
babel recall:  0.143747708104
initial_corpus size: 6351


(0.0, 0.0, 8547, 2727, 0.75)

In [160]:
bab = 'toxicology, Chemical toxicology, History of toxicology, Toxicological, Toxicologists'
synset = bab.split(',')
topic = 'Toxicology'
babelnet_eval(topic, synset)

initial_corpus size: 2550
results size of the topic "Toxicology": 5563
ground truth size 1552
intersection with the ground truth: 146


In [27]:
bab = 'toxicology, Chemical toxicology, History of toxicology, Toxicological, Toxicologists'
synset = bab.split(',')
topic = 'Toxicology'
res_pickleckleickle = "results/res__Toxicology"
eval_all_at_thresh_lst(res_pickle, topic, synset, thresh=0.75)

initial_corpus size: 2550
babelnet results size of the topic "Toxicology": 5563
ground truth size 1552
intersection with the ground truth: 146
babel precision:  0.0262448319252
babel recall:  0.0940721649485
initial_corpus size: 2550


(0.014391014391014392, 0.07925257731958762, 8547, 1552, 0.75)

In [162]:
bab = 'organ transplant, transplant, transplantation, Organ transplantation, Medical Transplantation, Transplant Surgery, Black market organs, First transplant, First transplantation, Intestinal transplant, Live organ transplants, Mixed chimerism, Organ and Tissue Donor, Organ doner, Organ farming, Organ transplantation in different countries, Organ transplantation therapy, Organ transplants, Skin transplant, Tissue transplant, Transplant Tourism and Organ Trafficking, Transplantation medicine, Transplantation surgery, Transplantation therapy, Transplanted organs, Transplantology'
synset = bab.split(',')
topic = 'Transplantation'
babelnet_eval(topic, synset)

initial_corpus size: 8999
results size of the topic "Transplantation": 8975
ground truth size 4997
intersection with the ground truth: 910


In [26]:
bab = 'organ transplant, transplant, transplantation, Organ transplantation, Medical Transplantation, Transplant Surgery, Black market organs, First transplant, First transplantation, Intestinal transplant, Live organ transplants, Mixed chimerism, Organ and Tissue Donor, Organ doner, Organ farming, Organ transplantation in different countries, Organ transplantation therapy, Organ transplants, Skin transplant, Tissue transplant, Transplant Tourism and Organ Trafficking, Transplantation medicine, Transplantation surgery, Transplantation therapy, Transplanted organs, Transplantology'
synset = bab.split(',')
topic = 'Transplantation'
res_pickle = "results/res_Transplantation"
eval_all_at_thresh_lst(res_pickle, topic, synset, thresh=0.75)

initial_corpus size: 8999
babelnet results size of the topic "Transplantation": 8975
ground truth size 4997
intersection with the ground truth: 910
babel precision:  0.10139275766
babel recall:  0.182109265559
initial_corpus size: 8999


(0.0848931288491728, 0.14068441064638784, 8281, 4997, 0.9)

In [163]:
# left becasue of small testset
bab = 'transit, transportation, public transport, transportation system, mass transit, Public transportation, Commuter transport, Government transportation, Mass-transit, Mass-transit system, Mass transit system, Mass transit systems, Mass transport, Mass transport system, Mass transportation, Mass transportation system, Pubic transportation, Public conveyance, Public mass transportation, public Transit, Public transit system, Public transport industry, Public transport service numbering, Public transport system, Public transportation system, Public utility vehicle, Public utility vehicles, Transit rider, Transit system, Transit Systems Management, Transit vehicle, Urban transit'
synset = bab.split(',')
topic = 'Transportation'
babelnet_eval(topic, synset)

initial_corpus size: 6180
results size of the topic "Transportation": 8297
ground truth size 359
intersection with the ground truth: 15


In [164]:
bab = 'tropical medicine, International medicine, Tropical Disease Research, Tropical pathology'
synset = bab.split(',')
topic = 'Tropical Medicine'
babelnet_eval(topic, synset)

initial_corpus size: 66
results size of the topic "Tropical Medicine": 78
ground truth size 94
intersection with the ground truth: 1


In [165]:
bab = 'Index of urban studies articles, Urban studies'
synset = bab.split(',')
topic = 'Urban Studies'
babelnet_eval(topic, synset)

initial_corpus size: 55
results size of the topic "Urban Studies": 55
ground truth size 10
intersection with the ground truth: 0


In [166]:
bab = 'veterinary medicine, Animal Health and Welfare, Veterinary science, Veterinary Sciences, Anatomy, veterinary, Animal diseases, Animal health, Animal health science, Animal Health Sciences, Animal hospital, Disease in animals, History of veterinary medicine, Pet hospice, Veterinary anatomy, Veterinary clinic, Veterinary hospice, Veterinary hospital, Veterinary scientist, Veterinary studies'
synset = bab.split(',')
topic = 'Veterinary Sciences'
babelnet_eval(topic, synset)

initial_corpus size: 6
results size of the topic "Veterinary Sciences": 8334
ground truth size 0
intersection with the ground truth: 0


In [167]:
bab = 'virology, virologist, Viriology, Virologists, Virulogy'
synset = bab.split(',')
topic = 'Virology'
babelnet_eval(topic, synset)

initial_corpus size: 456
results size of the topic "Virology": 485
ground truth size 231
intersection with the ground truth: 1


In [168]:
bab = 'assamesse, Water resources, Demand for water, Fresh water resources, Groundwater resources, Industrial water, Industrial water supply, Water, Water resource'
synset = bab.split(',')
topic = 'Water Resources'
babelnet_eval(topic, synset)

initial_corpus size: 1951
results size of the topic "Water Resources": 8128
ground truth size 251
intersection with the ground truth: 53


In [169]:
bab = "women's studies, feminist studies, Women studies, Women’s Studies, Criticisms of women's studies, Female studies, Weemyns studies, Wimmins studies, Woman's Studies, Woman studies, Womans studies, Womens' studies, Womens studies, Womins studies, Womyns studies"
synset = bab.split(',')
topic = "Women's Studies"
babelnet_eval(topic, synset)

initial_corpus size: 65
results size of the topic "Women's Studies": 108
ground truth size 4
intersection with the ground truth: 0


In [170]:
bab = 'zoology, zoological science, Animalogy, Branches of zoological study, Zooelogy, Zoography, Zoological, Zoologies, Zooology, Zoölogyi'
synset = bab.split(',')
topic = 'Zoology'
babelnet_eval(topic, synset)

initial_corpus size: 123
results size of the topic "Zoology": 877
ground truth size 10
intersection with the ground truth: 0


In [171]:
bab = 'social service, welfare work, Social work, Social care, Social workers, Care service, Careworker, Social affairs, Social care service, Social casework, Social Director, Social service program, Social service worker, Social Work Profession, social Worker, SocialWork, SOWK, Sozialarbeit'
synset = bab.split(',')
topic = 'Social Work'
babelnet_eval(topic, synset)

initial_corpus size: 2246
results size of the topic "Social Work": 5153
ground truth size 134
intersection with the ground truth: 81
